## Initializing Intrinsic Parameters and Distortion Coefficients

In [10]:
import json
import numpy as np

path_lcam_param = 'C:/_sw/_temp/stereo_left_1280_960.json'
path_rcam_param = 'C:/_sw/_temp/stereo_right_1280_960.json'

# Load the left camera parameters
with open(path_lcam_param, 'r') as f:
    data = json.load(f)
K_left = np.array(data['K'])
D_left = np.array(data['D']).reshape((4, 1))

# Load the right camera parameters
with open(path_rcam_param, 'r') as f:
    data = json.load(f)
K_right = np.array(data['K'])
D_right = np.array(data['D']).reshape((4, 1))


## Initializing Extrinsic Parameters

In [14]:
# Rotation matrix (assuming no rotation between the two cameras)
R = np.eye(3)  # Identity matrix for no rotation

# Translation vector (baseline = 65mm = 0.065 meters)
T = np.array([[-0.065], [0], [0]])

## Stereo Rectification
* Ensure that corresponding points in both the left and right images lie on the same horizontal line, making the disparity calculation easier.
* compute the rectification transforms.

In [16]:
import cv2

# Image size (adjust based on your images)
image_size = (1280, 960)

# Stereo rectification using OpenCV
# R1, R2 are the rectification transforms for the left and right images
# P1, P2 are the new projection matrices for the left and right images
# Q is the reprojection matrix used for 3D reconstruction (optional)
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(K_left, D_left, K_right, D_right, image_size, R, T)

# Compute rectification maps for left and right images
left_map1, left_map2 = cv2.initUndistortRectifyMap(K_left, D_left, R1, P1, image_size, cv2.CV_32FC1)
right_map1, right_map2 = cv2.initUndistortRectifyMap(K_right, D_right, R2, P2, image_size, cv2.CV_32FC1)

# Load the stereo images (left and right images)
left_image = cv2.imread('C:/_sw/ScriptVault/left_image_00000.png', cv2.IMREAD_GRAYSCALE)
right_image = cv2.imread('C:/_sw/ScriptVault/right_image_00000.png', cv2.IMREAD_GRAYSCALE)

# Apply the rectification maps to the images
rectified_left = cv2.remap(left_image, left_map1, left_map2, cv2.INTER_LINEAR)
rectified_right = cv2.remap(right_image, right_map1, right_map2, cv2.INTER_LINEAR)

# Display the rectified images
cv2.imshow("Rectified Left", rectified_left)
cv2.imshow("Rectified Right", rectified_right)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Disparity Map Calculation

In [20]:
import cv2
import numpy as np

# Load the rectified images
# rectified_left = cv2.imread('rectified_left.png', cv2.IMREAD_GRAYSCALE)
# rectified_right = cv2.imread('rectified_right.png', cv2.IMREAD_GRAYSCALE)

# Create StereoSGBM object
stereo_sgbm = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=16*5,  # Must be divisible by 16
    blockSize=5,
    P1=8*3*5**2,  # Control smoothness (smaller values = more detail)
    P2=32*3*5**2, # Larger value for more smooth disparity
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32
)

# Compute the disparity map
disparity = stereo_sgbm.compute(rectified_left, rectified_right)

# Normalize the disparity map for visualization
disparity_normalized = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Display the disparity map
cv2.imshow("Disparity Map - SGBM", disparity_normalized)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Depth Calculation

In [22]:
# Extract focal length (fx)
focal_length = K_left[0, 0]  # Focal length in pixels
baseline = 0.065  # Baseline in meters (65mm)

# Compute the depth map
depth_map = np.zeros(disparity.shape, dtype=np.float32)
depth_map[disparity > 0] = (focal_length * baseline) / disparity[disparity > 0]

# Normalize the depth map for visualization (optional)
depth_map_normalized = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Display the depth map
cv2.imshow("Depth Map", depth_map_normalized)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 3D Reconstruction

In [ ]:
# Reproject the disparity map to 3D space
points_3D = cv2.reprojectImageTo3D(disparity, Q)

# Mask to filter out points with invalid disparity (where disparity <= 0)
mask = disparity > 0

# Get the 3D points
output_points = points_3D[mask]

# Get the colors for each point from the rectified left image (for visualization)
colors = cv2.cvtColor(rectified_left, cv2.COLOR_GRAY2RGB)
output_colors = colors[mask]

# Now you have a 3D point cloud in `output_points` and the corresponding colors in `output_colors`

# Optionally, export the point cloud to a file for visualization in other software (e.g., PLY format)
def write_ply(filename, vertices, colors):
    vertices = vertices.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    with open(filename, 'w') as f:
        f.write(f"ply\nformat ascii 1.0\nelement vertex {len(vertices)}\n")
        f.write("property float x\nproperty float y\nproperty float z\n")
        f.write("property uchar red\nproperty uchar green\nproperty uchar blue\n")
        f.write("end_header\n")
        for i in range(len(vertices)):
            f.write(f"{vertices[i, 0]} {vertices[i, 1]} {vertices[i, 2]} {colors[i, 0]} {colors[i, 1]} {colors[i, 2]}\n")

# Save the point cloud to a PLY file
write_ply("output_point_cloud.ply", output_points, output_colors)

print("3D point cloud saved to 'output_point_cloud.ply'")